# ERC-4626: list all vaults

This notebook serves both as a coding tutorial and a useful data analytics tool for ERC-4626 vaults. 

- In this notebook, we examine different ERC-4626 vaults across different EVM blockchains   
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

In this notebook, we use terms Net Asset Value (NAV) and [Total Value Locked (TVL)](https://tradingstrategy.ai/glossary/total-value-locked-tvl) interchangeably.

## Usage

- Read general instructions [how to run the tutorials](./)
- See `ERC-4626 scanning all vaults onchain` example in tutorials first how to build a vault database as local `vault_db.pickle` file.




## Setup

- Set up notebook renderinb parmaeters

In [1]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [2]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 8526 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [3]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like

data = list(vault_db.values())
df = pd.DataFrame(data)

# print("Raw row example:")
# print(df.iloc[0])

# Build useful columns out of raw pickled Python data
# _detection_data contains entries as ERC4262VaultDetection class
entry: ERC4262VaultDetection
df["Chain"] = df["_detection_data"].apply(lambda entry: get_chain_name(entry.chain))
df["Protocol identified"] = df["_detection_data"].apply(lambda entry: entry.is_protocol_identifiable())
df["Stablecoin denominated"] = df["_denomination_token"].apply(lambda token_data: is_stablecoin_like(token_data["symbol"]) if pd.notna(token_data) else False)
df["ERC-7540"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7540())
df["ERC-7575"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7575())
df["Fee detected"] = df.apply(lambda row: (row["Mgmt fee"] is not None) or (row["Perf fee"] is not None), axis=1)
# Event counts
df["Deposit count"] = df["_detection_data"].apply(lambda entry: entry.deposit_count)
df["Redeem count"] = df["_detection_data"].apply(lambda entry: entry.redeem_count)
df["Total events"] = df["Deposit count"] + df["Redeem count"]
df["Mgmt fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Perf fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Age"] = datetime.datetime.utcnow() - df["First seen"]
df["NAV"] = df["NAV"].astype("float64")

df = df.sort_values(["Chain", "Address"])
df = df.set_index(["Chain", "Address"])

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

source_df = df

display(df.sort_values("Total events", ascending=False).head())



DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Denomination, NAV, Protocol, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, ERC-7575, Fee detected, Deposit count, Redeem count, Total events, Age


Symbol  \
Chain     Address                                                
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a   UB-WETH   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701      sDAI   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  yvUSDC-A   
          0xbb287e6017d3deb0e2e65061e8684eab21060123  yvUSDT-A   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2    BBiBGT   

                                                                 Name  \
Chain     Address                                                       
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a          UB-WETH   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701     Savings xDAI   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0    USDC yVault-A   
          0xbb287e6017d3deb0e2e65061e8684eab21060123    USDT yVault-A   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2  Beraborrow iBGT   

                                                     Denomination  \
Chain     Address                                                   
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a         WETH   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701        WXDAI   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0         USDC   
          0xbb287e6017d3deb0e2e65061e8684eab21060123         USDT   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2         iBGT   

                                                               NAV  \
Chain     Address                                                    
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a        372.03   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701 82,539,877.94   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0    549,657.25   
          0xbb287e6017d3deb0e2e65061e8684eab21060123    411,832.29   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2    218,826.03   

                                                                Protocol  \
Chain     Address                                                          
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a  <unknown ERC-4626>   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701  <unknown ERC-4626>   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0            Yearn v3   
          0xbb287e6017d3deb0e2e65061e8684eab21060123            Yearn v3   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2  <unknown ERC-4626>   

                                                       Mgmt fee   Perf fee  \
Chain     Address                                                            
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a  <unknown>  <unknown>   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701  <unknown>  <unknown>   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  <unknown>  <unknown>   
          0xbb287e6017d3deb0e2e65061e8684eab21060123  <unknown>  <unknown>   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2  <unknown>  <unknown>   

                                                                           Shares  \
Chain     Address                                                                   
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a       370.590545324542904394   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701  70162919.227732298114111541   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0                465536.300123   
          0xbb287e6017d3deb0e2e65061e8684eab21060123                346421.293716   
Berachain 0xe59ab0c3788217e48399dae3cd11929789e4d3b2    163229.490885064843638595   

                                                              First seen  \
Chain     Address                                                          
Soneium   0x232554b4b291a446b4829300bec133fbb07a8f2a 2025-01-13 06:53:05   
Gnosis    0xaf204776c7245bf4147c2612bf6e5972ee483701 2023-09-28 12:13:30   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0 2023-10-26 16:45:48   
          0xbb287e6017d3deb0e2e65061e8684eab210601

## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [4]:
nav_threshold = 100_000
broken_max_nav_threshold = 999_000_000_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]

# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby(level='Chain').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby(level='Chain')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby(level='Chain').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby(level='Chain').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby(level='Chain').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby(level='Chain').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby(level='Chain').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby(level='Chain').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Chain,,,,,,,,
Arbitrum,1960,"169,933,937.56",350.00,679,57.00,19.00,27.00,221.00
Avalanche,310,"214,255,450.65",44.00,89,10.00,1.00,1.00,48.00
Base,1340,"407,955,546.79",624.00,510,62.00,38.00,17.00,114.00
Berachain,484,"189,979,301.20",46.00,26,14.00,0.00,0.00,5.00
Binance,361,"272,895,412.40",49.00,106,24.00,0.00,0.00,27.00
Blast,106,"1,954,028.20",2.00,14,2.00,0.00,0.00,32.00
Celo,9,"3,196,291.28",0.00,6,1.00,0.00,0.00,0.00
Ethereum,2214,"10,787,586,025.98",567.00,772,223.00,19.00,17.00,192.00
Gnosis,89,"85,149,495.94",13.00,37,5.00,0.00,0.00,0.00


## Vaults per protocol summary

- Break down by identified protocol
- Because of the brokeness of EVM, Solidity and smart contract development practices, protocol-rules are hand-maintained heurestics and there can false positives and negatives

In [5]:
df = source_df.copy()
df = df.reset_index()
# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]
# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby('Protocol').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby('Protocol')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby('Protocol').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby('Protocol').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby('Protocol').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby('Protocol').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby('Protocol').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby('Protocol').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
summary_df = summary_df.sort_values("Total vault TVL/NAV in stable vaults", ascending=False)
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Protocol,,,,,,,,
<unknown ERC-4626>,6127,"9,757,068,782.87",0.00,"2,152.00",240.00,0.00,12.00,0.00
Morpho,204,"1,434,631,831.29",204.00,118.00,67.00,0.00,0.00,204.00
Fluid,59,"481,322,896.90",59.00,26.00,8.00,0.00,0.00,0.00
Euler Vault Kit,323,"359,369,132.36",323.00,94.00,37.00,0.00,0.00,0.00
Yearn v3,139,"82,263,373.87",139.00,69.00,20.00,0.00,0.00,0.00
Kiln Metavault,76,"40,990,585.66",76.00,67.00,24.00,0.00,0.00,0.00
Superform,140,"25,421,608.97",140.00,57.00,11.00,20.00,0.00,0.00
Gains Network,39,"10,756,199.17",39.00,17.00,10.00,3.00,0.00,0.00
Term Finance,16,"10,214,572.96",16.00,15.00,5.00,0.00,0.00,0.00


## Vault deployment history

- Show how much history we have for each chain


In [6]:
# Assuming your DataFrame is named 'df'
df = source_df
seen_df = df.groupby(level='Chain')['First seen'].agg(['min', 'max']).reset_index()

# Rename columns for clarity
seen_df.columns = ['Chain', 'First vault deployed', 'Last vault deployed']

seen_df = seen_df.set_index("Chain")

display(seen_df)

,First vault deployed,Last vault deployed
Chain,,
Arbitrum,2022-03-28 09:03:11,2025-05-02 10:20:31
Avalanche,2022-04-11 00:05:24,2025-05-05 14:40:43
Base,2023-08-04 15:36:07,2025-05-08 11:26:55
Berachain,2025-01-26 00:06:03,2025-05-08 05:48:40
Binance,2022-05-27 17:25:18,2025-05-08 02:32:06
Blast,2024-02-29 20:15:37,2024-12-03 15:38:15
Celo,2022-08-08 21:14:10,2025-04-29 19:46:04
Ethereum,2019-06-11 06:17:19,2025-05-06 13:48:23
Gnosis,2022-05-01 18:47:35,2025-04-29 10:03:35


## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [7]:
largest_threshold = 20
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100] 
largest_df = largest_df[largest_df["Stablecoin denominated"] == True] 
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["NAV", "Chain", "Address", "Name", "Denomination", "Total events", "Mgmt fee", "Perf fee"]]
largest_df = largest_df.set_index("Name")


display(largest_df.head(largest_threshold))


,NAV,Chain,Address,Denomination,Total events,Mgmt fee,Perf fee
Name,,,,,,,
Savings USDS,"3,209,609,512.52",Ethereum,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,32940,<unknown>,<unknown>
Staked USDe,"2,026,581,476.34",Ethereum,0x9d39a5de30e57443bff2a8307a4256c8797a3497,USDe,70579,<unknown>,<unknown>
Ethereal Pre-deposit Vault,"1,098,908,718.40",Ethereum,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,USDe,61124,<unknown>,<unknown>
Savings Dai,"562,754,348.75",Ethereum,0x83f20f44975d03b1b09e64809b757c47f942beea,DAI,74291,<unknown>,<unknown>
Syrup USDC,"298,863,687.35",Ethereum,0x80ac24aa929eaf5013f6436cda2a7ba190f5cc0b,USDC,5342,<unknown>,<unknown>
Spark USDC Vault,"284,041,067.03",Ethereum,0xbc65ad17c5c0a2a4d159fa5a503f4992c7b545fe,USDC,1341,<unknown>,<unknown>
Staked Falcon USD,"224,164,916.45",Ethereum,0xc8cf6d7991f15525488b2a83df53468d682ba4b0,USDf,3822,<unknown>,<unknown>
Fluid USD Coin,"210,361,928.66",Ethereum,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,USDC,18987,<unknown>,<unknown>
Staked USDX,"203,278,241.97",Binance,0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,USDX,5595,<unknown>,<unknown>


## Largest USD vault per chain

- Get the largest vault of each chain

In [8]:
# Get the index of max NAV for each chain
largest_df = largest_df.reset_index().set_index(["Chain", "Name"])
max_nav_idx = largest_df.groupby('Chain')['NAV'].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]

display(result)

NAV  \
Chain       Name                                         
Arbitrum    ostiumLP                     63,204,342.63   
Avalanche   EVK Vault eUSDC-2            69,096,020.22   
Base        Spark USDC Vault            166,451,248.48   
            Spark USDC Vault             14,120,271.56   
Berachain   Concrete-USDe-Vault          71,948,602.11   
Binance     Staked USDX                 203,278,241.97   
Blast       Wasabi USDB Vault             1,452,788.90   
Celo        Staked EURA                   3,196,286.58   
Ethereum    Savings USDS              3,209,609,512.52   
Gnosis      Savings xDAI                 82,539,877.94   
Hyperliquid hyUSDXL (Looped HYPE) - 1     2,173,111.13   
Mode        USDC Ironclad Vault             337,647.27   
Optimism    exactly USDC                  3,637,150.38   
Polygon     Compound USDC                24,832,880.33   
Soneium     UB-USDC                       1,173,465.19   
ZKsync      hsUSDC                          707,286.34   

                                                                          Address  \
Chain       Name                                                                    
Arbitrum    ostiumLP                   0x20d419a8e12c45f88fda7c5760bb6923cee27f98   
Avalanche   EVK Vault eUSDC-2          0x39de0f00189306062d79edec6dca5bb6bfd108f9   
Base        Spark USDC Vault           0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a   
            Spark USDC Vault           0x3128a0f7f0ea68e7b7c9b00afa7e41045828e858   
Berachain   Concrete-USDe-Vault        0x59e24f42cae1b82c8b2dc79ea898f2f8b4986dfc   
Binance     Staked USDX                0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92   
Blast       Wasabi USDB Vault          0x4bed2a922654cacc2be974689619768fabf24855   
Celo        Staked EURA                0x004626a008b1acdc4c74ab51644093b155e59a23   
Ethereum    Savings USDS               0xa3931d71877c0e7a3148cb7eb4463524fec27fbd   
Gnosis      Savings xDAI               0xaf204776c7245bf4147c2612bf6e5972ee483701   
Hyperliquid hyUSDXL (Looped HYPE) - 1  0xaeedd5b6d42e0f077ccf3e7a78ff70b8cb217329   
Mode        USDC Ironclad Vault        0x882fd369341fc435ad5e54e91d1ebc23b1fc6d4c   
Optimism    exactly USDC               0x6926b434cce9b5b7966ae1bfeef6d0a7dcf3a8bb   
Polygon     Compound USDC              0x781fb7f6d845e3be129289833b04d43aa8558c42   
Soneium     UB-USDC                    0xc675bb95d73ca7db2c09c3dc04daaa7944ccba41   
ZKsync      hsUSDC                     0xaf08a9d918f16332f22cf8dc9abe9d9e14ddcbc2   

                                      Denomination  Total events   Mgmt fee  \
Chain       Name                                                              
Arbitrum    ostiumLP                          USDC          7332  <unknown>   
Avalanche   EVK Vault eUSDC-2                 USDC          3815  <unknown>   
Base        Spark USDC Vault                  USDC         13292       0.00   
            Spark USDC Vault                  USDC         37052  <unknown>   
Berachain   Concrete-USDe-Vault               USDe           207  <unknown>   
Binance     Staked USDX                       USDX          5595  <unknown>   
Blast       Wasabi USDB Vault                 USDB         92958  <unknown>   
Celo        Staked EURA                       EURA           225  <unknown>   
Ethereum    Savings USDS                      USDS         32940  <unknown>   
Gnosis      Savings xDAI                     WXDAI        616549  <unknown>   
Hyperliquid hyUSDXL (Looped HYPE) - 1        USDXL           798  <unknown>   
Mode        USDC Ironclad Vault               USDC           183  <unknown>   
Optimism    exactly USDC                      USDC         68228  <unknown>   
Polygon     Compound USDC                     USDC          6388       0.00   
Soneium     UB-USDC                         USDC.e        242191  <unknown>   
ZKsync      hsUSDC                          USDC.e         48739  <unknown>   

                                        Perf fee  
C

## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [9]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold))

,Total events,Chain,Address,Denomination,NAV,Age,Deposit count,Redeem count
Name,,,,,,,,
UB-WETH,736121,Soneium,0x232554b4b291a446b4829300bec133fbb07a8f2a,WETH,372.03,120 days 12:10:33.955174,540971,195150
Savings xDAI,616549,Gnosis,0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"82,539,877.94",593 days 06:50:08.955174,333916,282633
USDC yVault-A,490304,Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,USDC,"549,657.25",565 days 02:17:50.955174,465215,25089
USDT yVault-A,458355,Polygon,0xbb287e6017d3deb0e2e65061e8684eab21060123,USDT,"411,832.29",523 days 02:01:18.955174,407726,50629
Beraborrow iBGT,409835,Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"218,826.03",67 days 06:58:19.955174,211085,198750
BathToken v1,297336,Optimism,0xe0e112e8f33d3f437d1f895cbb1a456836125952,USDC,"60,446.49",1084 days 19:41:34.955174,150209,147127
Wasabi WETH Vault,243139,Blast,0x8e2b50413a53f50e2a059142a9be060294961e40,WETH,828.14,438 days 22:48:01.955174,161254,81885
UB-USDC,242191,Soneium,0xc675bb95d73ca7db2c09c3dc04daaa7944ccba41,USDC.e,"1,173,465.19",120 days 12:11:27.955174,172168,70023
Bridged USDC (Stargate)Vault,239669,Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,0.39,107 days 18:57:35.955174,110379,129290


## Most historically active vault per chain

- Vaults with most deposit and redeem events

In [10]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby('Chain')['Total events'].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

result = result.set_index(["Chain", "Name"])

display(result)

,,Total events,Address,Denomination,NAV,Age,Deposit count,Redeem count
Chain,Name,,,,,,,
Arbitrum,Staked USDA,"122,295.00",0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"908,852.09",488 days 10:31:13.955174,120071,2224
Avalanche,GoGoPool Liquid Staking Token,"79,994.00",0xa25eaf2906fa1a3a13edac9b9657108af7b703e3,WAVAX,"906,421.20",768 days 21:52:53.955174,47824,32170
Base,Moonwell Flagship ETH,"147,810.00",0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"12,606.43",335 days 06:06:23.955174,88644,59166
Berachain,Beraborrow iBGT,"409,835.00",0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"218,826.03",67 days 06:58:19.955174,211085,198750
Binance,kUSDT,"43,021.00",0x1c3f35f7883fc4ea8c4bca1507144dc6087ad0fb,VUSD,"1,518,589.90",701 days 11:54:55.955174,25369,17652
Blast,Wasabi WETH Vault,"243,139.00",0x8e2b50413a53f50e2a059142a9be060294961e40,WETH,828.14,438 days 22:48:01.955174,161254,81885
Celo,Wrapped mcUSD,241.00,0x5f71fe197fd9aad95b7b5510a98882769622112a,mcUSD,"32,063.28",970 days 23:29:41.955174,180,61
Ethereum,pufETH,"168,577.00",0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"61,073.10",467 days 23:19:03.955174,151964,16613
Gnosis,Savings xDAI,"616,549.00",0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"82,539,877.94",593 days 06:50:08.955174,333916,282633


## Oldest vaults

- Show oldest vaults

In [11]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby('Chain')['Age'].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

result = result.set_index("Chain")

display(result)

,Address,Name,Age,Denomination,NAV,Total events
Chain,,,,,,
Arbitrum,0xf46ce0c13577232d5f29d9bd78a9cab278755346,Jones ETH,1082 days 03:48:32.955174,WETH,17.99,"2,338.00"
Avalanche,0x9dd17f32fc8355ae37425f475a10cc7bec8ca36a,,1126 days 02:06:16.955174,,0.00,"1,425.00"
Base,0xc7548d8d7560f6679e369d0556c44fe1eddea3e9,FARM_WETH,621 days 03:09:45.955174,WETH,0.97,"1,119.00"
Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,Bridged USDC (Stargate)Vault,107 days 18:57:35.955174,USDC.e,0.39,"239,669.00"
Binance,0x0f8754b36a767c5579178bd8a04d2fcd9d530b70,ygNRCH,1050 days 22:10:59.955174,NRCH,"1,188,874.57","1,196.00"
Blast,0x8e2b50413a53f50e2a059142a9be060294961e40,Wasabi WETH Vault,438 days 22:48:01.955174,WETH,828.14,"243,139.00"
Ethereum,0x815c23eca83261b6ec689b60cc4a58b54bc24d8d,vTHOR,1114 days 07:38:01.955174,THOR,"88,513,877.38","19,836.00"
Gnosis,0x004626a008b1acdc4c74ab51644093b155e59a23,Staked EURA,620 days 05:21:43.955174,EURA,"522,895.79","3,631.00"
Hyperliquid,0x2831775cb5e64b1d892853893858a261e898fbeb,wHYPE,61 days 20:52:38.955174,WHYPE,"385,863.72","2,222.00"


## All vaults

- List of all stablecoin vaults

In [12]:
all_df = df.reset_index()

min_nav = 10_000

stablecoin_denominated = all_df["Stablecoin denominated"] == True
all_df = all_df[stablecoin_denominated]
all_df = all_df[all_df["NAV"] < broken_max_nav_threshold]
all_df = all_df[all_df["NAV"] >= min_nav]
all_df = all_df[["NAV", "Chain", "Protocol", "Address", "Name", "Age", "Denomination", "Total events"]]
all_df = all_df.set_index("Name").sort_values("NAV", ascending=False)

print(f"Displaying {len(all_df)} stablecoin vaults w/min NAV of {min_nav} USD")
pd.options.display.max_rows = None
display(all_df)


Displaying 632 stablecoin vaults w/min NAV of 10000 USD


,NAV,Chain,Protocol,Address,Age,Denomination,Total events
Name,,,,,,,
Savings USDS,"3,209,609,512.52",Ethereum,<unknown ERC-4626>,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,238 days 05:26:51.955174,USDS,32940
Staked USDe,"2,026,581,476.34",Ethereum,<unknown ERC-4626>,0x9d39a5de30e57443bff2a8307a4256c8797a3497,539 days 10:57:27.955174,USDe,70579
Ethereal Pre-deposit Vault,"1,098,908,718.40",Ethereum,<unknown ERC-4626>,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,89 days 03:49:27.955174,USDe,61124
Savings Dai,"562,754,348.75",Ethereum,<unknown ERC-4626>,0x83f20f44975d03b1b09e64809b757c47f942beea,776 days 08:38:51.955174,DAI,74291
Spark DAI Vault,"300,098,440.19",Ethereum,Morpho,0x73e65dbd630f90604062f6e02fab9138e713edd9,410 days 04:43:51.955174,DAI,100
Syrup USDC,"298,863,687.35",Ethereum,<unknown ERC-4626>,0x80ac24aa929eaf5013f6436cda2a7ba190f5cc0b,332 days 00:32:27.955174,USDC,5342
Spark USDC Vault,"284,041,067.03",Ethereum,<unknown ERC-4626>,0xbc65ad17c5c0a2a4d159fa5a503f4992c7b545fe,69 days 04:03:27.955174,USDC,1341
Staked Falcon USD,"224,164,916.45",Ethereum,<unknown ERC-4626>,0xc8cf6d7991f15525488b2a83df53468d682ba4b0,89 days 22:10:39.955174,USDf,3822
Fluid USD Coin,"210,361,928.66",Ethereum,Fluid,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,449 days 17:48:15.955174,USDC,18987
